In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from pathlib import Path
from fastai.vision.all import *
from fastai.metrics import error_rate

In [3]:
images_folder = "yolo-focused-3"
masks_folder = "focused_masks-3"
path = Path(f'/home/p.kuznetsov/runaway/datasets')

In [4]:
def get_mask_path(image_path):
    return str(image_path).replace(images_folder, masks_folder).replace("/images", "").replace(".jpg", ".png")

In [5]:
segmentation_block = DataBlock(
    blocks=(ImageBlock, MaskBlock),
    get_items=get_image_files,
    splitter=GrandparentSplitter(train_name='train', valid_name='valid'),
    get_y=get_mask_path,
)

In [6]:
dls = segmentation_block.dataloaders(path/images_folder, path=path, bs=4)  

In [7]:
model = "resnet50"
learn = unet_learner(dls, resnet50, metrics=Dice(), n_out=3)
learn.opt_fn = Adam

In [ ]:
learn.fit(1, 5e-4)

In [ ]:
learn.save(f"{model}-stage-1")

In [ ]:
learn.load(f"{model}-stage-1")

In [ ]:
learn.fit_one_cycle(1, slice(1e-5, 1e-3))

In [ ]:
learn.save(f"{model}-stage-2")

In [ ]:
learn.load(f"{model}-stage-2")

In [ ]:
learn.fit_one_cycle(1, slice(1e-5, 1e-3))

In [ ]:
learn.save(f"{model}-stage-3")

In [ ]:
learn.load(f"{model}-stage-3")

In [ ]:
learn.fit_one_cycle(1, slice(1e-5, 1e-3))

In [ ]:
learn.save(f"{model}-stage-4")

In [17]:
learn.load(f"{model}-stage-2")

In [18]:
learn.validate()

(#2) [0.01408215519040823,0.622025286754686]